In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [ ]:
    configs = {"fs.azure.account.auth.type": "OAuth",
                "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
                "fs.azure.account.oauth2.client.id": "",
                "fs.azure.account.oauth2.client.secret": '',
                "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tanent_id/oauth2/token"}


dbutils.fs.mount(
source = "abfss://tokyo-data@tokyoolympicdatakao.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/tokyoolymic-test",
extra_configs = configs)

In [ ]:
%fs
ls "/mnt/tokyoolymic-test"

In [ ]:
#Using Spark to the data from folder

athletes = spark.read.format("csv").option('header', 'true').option("inferSchema", "true").load("/mnt/tokyoolymic-test/raw-data/athletes.csv")
coaches = spark.read.format("csv").option('header', 'true').option("inferSchema", "true").load("/mnt/tokyoolymic-test/raw-data/coaches.csv")
entriesgender = spark.read.format("csv").option('header', 'true').option("inferSchema", "true").load("/mnt/tokyoolymic-test/raw-data/entriesgender.csv")
medals = spark.read.format("csv").option('header', 'true').option("inferSchema", "true").load("/mnt/tokyoolymic-test/raw-data/medals.csv")
teams = spark.read.format("csv").option('header', 'true').option("inferSchema", "true").load("/mnt/tokyoolymic-test/raw-data/teams.csv")

#option('inferSchema', 'true') to make a datatype in the table precise

Data Inspectaion

In [ ]:
athletes.show()

In [ ]:
athletes.printSchema()

In [ ]:
coaches.show()

In [ ]:
coaches.printSchema()

In [ ]:
entriesgender.show()

In [ ]:
entriesgender.printSchema()

In [ ]:
#Transform the data type manual way

entriesgender = entriesgender.withColumn("Female", col("Female").cast(IntegerType()))\
    .withColumn("Male", col("Male").cast(IntegerType()))\
        .withColumn("Total", col("Total").cast(IntegerType()))

#Do the transform manual way

In [ ]:
entriesgender.printSchema()

In [ ]:
medals.show()

In [ ]:
medals.printSchema()

In [ ]:
teams.show()

In [ ]:
teams.printSchema()

In [ ]:
#Find top countries with the heightest of gold medals
top_gold_countries = medals.orderBy("Gold", ascending=False).select("Team_country", "Gold").show()

In [ ]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesgender.withColumn(
    'Avg_Female', entriesgender['Female'] / entriesgender['Total']
).withColumn(
    'Avg_Male', entriesgender['Male'] / entriesgender['Total']
)
average_entries_by_gender.show()

In [ ]:
#Svae the file back into Azuer datalake
athletes.repartition(1).write.mode("overwrite").option('header', 'true').csv('/mnt/tokyoolymic-test/transformed-data/athletes')
coaches.repartition(1).write.mode("overwrite").option('header', 'true').csv('/mnt/tokyoolymic-test/transformed-data/coaches')
entriesgender.repartition(1).write.mode("overwrite").option('header', 'true').csv('/mnt/tokyoolymic-test/transformed-data/entriesgender')
medals.repartition(1).write.mode("overwrite").option('header', 'true').csv('/mnt/tokyoolymic-test/transformed-data/medals')
teams.repartition(1).write.mode("overwrite").option('header', 'true').csv('/mnt/tokyoolymic-test/transformed-data/teams')